# Week 17, Lecture 02: CodeAlong


## 🕹️Part 2: Explaining Models with Model Explainers

### Lesson Objectives

- By the end of this lesson, students will be able to:
    - Load variables and models from a joblib file into a new notebook.
    - Apply permutation importance
    - Apply shap analysis 
    - Visualize global and local explanations.


### Continuing with Life Expectancy Prediction

> Task Inspired by: https://medium.com/@shanzehhaji/using-a-linear-regression-model-to-predict-life-expectancy-de3aef66ac21

- Kaggle Dataset on Life Expectancy:
    - https://www.kaggle.com/datasets/kumarajarshi/life-expectancy-who

In [ ]:
## Our standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as miss

## Preprocessing tools
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

## Models & evaluation metrics
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor


## setting random state for reproducibility
SEED = 321
np.random.seed(SEED)
## Matplotlib style
fav_style = ('ggplot','tableau-colorblind10')
fav_context  ={'context':'notebook', 'font_scale':1.1}
plt.style.use(fav_style)
sns.set_context(**fav_context)
plt.rcParams['savefig.transparent'] = False
plt.rcParams['savefig.bbox'] = 'tight'


import joblib, os

In [ ]:
## Importing Custom Functions
import sys,os
# sys.path.append(os.path.abspath("../"))
%load_ext autoreload
%autoreload 2
from CODE import data_enrichment as de

### Functionized Code From Part 1

In [ ]:
def evaluate_regression(model, X_train,y_train, X_test, y_test,for_slides=True): 
    """Evaluates a scikit learn regression model using r-squared and RMSE
    FOR SLIDES VERS DOES MULTIPLE PRINT STATEMENTS FOR VERTICAL DISPLAY OF INFO"""
    
    ## Training Data
    y_pred_train = model.predict(X_train)
    r2_train = metrics.r2_score(y_train, y_pred_train)
    rmse_train = metrics.mean_squared_error(y_train, y_pred_train, 
                                            squared=False)
    mae_train = metrics.mean_absolute_error(y_train, y_pred_train)
    

    ## Test Data
    y_pred_test = model.predict(X_test)
    r2_test = metrics.r2_score(y_test, y_pred_test)
    rmse_test = metrics.mean_squared_error(y_test, y_pred_test, 
                                            squared=False)
    mae_test = metrics.mean_absolute_error(y_test, y_pred_test)
    
    if for_slides:
        df_version =[['Split','R^2','MAE','RMSE']]
        df_version.append(['Train',r2_train, mae_train, rmse_train])
        df_version.append(['Test',r2_test, mae_test, rmse_test])
        df_results = pd.DataFrame(df_version[1:], columns=df_version[0])
        df_results = df_results.round(2)
        display(df_results.style.hide(axis='index').format(precision=2, thousands=','))
        
    else: 
        print(f"Training Data:\tR^2 = {r2_train:,.2f}\tRMSE = {rmse_train:,.2f}\tMAE = {mae_train:,.2f}")
        print(f"Test Data:\tR^2 = {r2_test:,.2f}\tRMSE = {rmse_test:,.2f}\tMAE = {mae_test:,.2f}")

def get_coefficients(lin_reg):
    coeffs = pd.Series(lin_reg.coef_, index= lin_reg.feature_names_in_)
    coeffs.loc['intercept'] = lin_reg.intercept_
    return coeffs

def plot_coefficients(coeffs, sort_values=True, top_n=None, figsize=(6,4),
                     title="Linear Regression Coefficients", xlabel='Coefficient'):
    """Plots a Series of coefficients as horizotal bar chart, with option to sort
    and to only keep top_n coefficients"""
        
    if top_n is not None:
        top_n = coeffs.abs().rank().sort_values(ascending=False).head(top_n)
        coeffs = coeffs.loc[top_n.index]
        
    if sort_values:
        coeffs = coeffs.sort_values()

        
        
    ax = coeffs.plot(kind='barh', figsize=figsize)
    ax.axvline(0, color='k')
    ax.set(xlabel=xlabel, title=title);
    plt.show()
    return ax


def get_importances(rf_reg):
    importances = pd.Series(rf_reg.feature_importances_, index= rf_reg.feature_names_in_)
    return importances


def plot_importances(importances, sort_values=True, top_n=None, figsize=(6,4),
                     title="Feature Importance", xlabel='Importance'):
    if sort_values:
        importances = importances.sort_values()
        
    if top_n is not None:
        importances = importances.tail(top_n)
        
        
    ax = importances.plot(kind='barh', figsize=figsize)
    ax.axvline(0, color='k')
    ax.set(xlabel=xlabel, title=title);
    plt.show()
    return ax

# Loading Objects from a Joblib

In [ ]:
fname = "Models/wk1-lect01-codealong.joblib"
loaded = joblib.load(fname)
loaded.keys()

In [ ]:
X_train = loaded['X_train']
X_test = loaded['X_test']
y_train = loaded['y_train']
y_test = loaded['y_test']

preprocessor = loaded['preprocessor']
lin_reg = loaded['LinearRegression']
rf_reg = loaded['RandomForestRegressor']

> Let's evaluate our models to prove they saved correctly.

In [ ]:
evaluate_regression(lin_reg,X_train,y_train, X_test, y_test)

> ***Q: what happened??***

In [ ]:
## let's check X_train
X_train.head()

> **Q: What is missing/wrong?**
....

...


### Re-Creating X_train_df & X_test_df

In [ ]:
feature_names = preprocessor.get_feature_names_out()
## Using the already-fit preprocessor to recreate the vars
X_train_df = pd.DataFrame(preprocessor.transform(X_train),
                          index=X_train.index,
                          columns=feature_names)

X_test_df = pd.DataFrame(preprocessor.transform(X_test),
                         index=X_test.index,
                          columns=feature_names)
X_train_df.head()

### Evaluating Our LinearRegression

In [ ]:
evaluate_regression(lin_reg,X_train_df,y_train, X_test_df, y_test)

In [ ]:
pd.set_option('display.float_format',lambda x: f"{x:,.2f}")

In [ ]:
coeffs = get_coefficients(lin_reg)
coeffs

In [ ]:
plot_coefficients(coeffs)

### Evaluating Our Random Forest

In [ ]:
evaluate_regression(rf_reg,X_train_df,y_train, X_test_df, y_test)
importances = get_importances(rf_reg)
plot_importances(importances)

# Permutation Importance

In [ ]:
from sklearn.inspection import permutation_importance

In [ ]:
%%time
result = permutation_importance(rf_reg, X_test_df, y_test,scoring='r2', n_repeats=3,
                                n_jobs=-1,random_state=SEED)
result.keys()

In [ ]:
perm_importances = pd.Series(result['importances_mean'], index=rf_reg.feature_names_in_)
perm_importances

In [ ]:
plot_importances(perm_importances,title='Permutation Importance')

In [ ]:
plot_importances(importances)

### Permutation Importance Can Be Applied to ANY Model

In [ ]:
results_linreg = permutation_importance(lin_reg, X_test_df, y_test,scoring='r2', n_repeats=3,
                                n_jobs=-1,random_state=SEED)
result.keys()

In [ ]:
def get_permutation_importance(rf_reg,X_test_df,y_test, scoring='r2',
                               n_repeats=3, n_jobs=-1,random_state=SEED):

    result = permutation_importance(rf_reg, X_test_df, y_test,scoring=scoring, 
                                    n_repeats=n_repeats, n_jobs=n_jobs,
                                    random_state=random_state)
    perm_importances = pd.Series(result['importances_mean'], index=rf_reg.feature_names_in_)
    return perm_importances

In [ ]:
perm_importances_linreg = get_permutation_importance(lin_reg, X_test_df, y_test)
plot_importances(perm_importances_linreg)

In [ ]:
final_plot_df = pd.concat([X_train_df, y_train], axis=1)

In [ ]:
corr = final_plot_df.corr()
corr

In [ ]:
corr['Life expectancy'].sort_values(ascending=False).to_frame().style.bar()

# Global Model Explanations

##  Shap (For Regression)

In [ ]:
# Import and init shap
import shap
shap.initjs()

In [ ]:
# Take a sample of the training data
X_shap = shap.sample(X_train_df,nsamples = 500,random_state=SEED)
y_shap = y_train.loc[X_shap.index]
X_shap.head()

In [ ]:
# Instantiate a Model Explainer with the model
explainer = shap.Explainer(rf_reg)

## Get shap values form the explainer
shap_values = explainer(X_shap,y_shap)

In [ ]:
shap.summary_plot(shap_values, features = X_shap)

In [ ]:
explainer_linreg = shap.Explainer(lin_reg, X_shap)
shap_values_linreg = explainer_linreg(X_shap)
shap_values_linreg.shape

In [ ]:
shap.summary_plot(shap_values_linreg, features = X_shap)

# Individual Explanations

## Shap Force Plot

> So why is our LinReg predicting a high life expectancy when infant deaths are high?

In [ ]:
# shap_values_linreg[2643]

In [ ]:
## Making a vers of shap vars with 0-based integer index 
X_shap_local = X_shap.reset_index(drop=True)
y_shap_local = y_shap.reset_index(drop=True)
X_shap_local.head()

In [ ]:
# what is the max/range of infant deaths
X_shap_local['infant deaths'].describe()

In [ ]:
## saving the index of the most deaths
idx_high_deaths = X_shap_local['infant deaths'].idxmax()
idx_high_deaths

In [ ]:
# checking the feature values for selected example
X_shap_local.iloc[idx_high_deaths]

In [ ]:
## what was the actual life expectancy?
y_shap_local.iloc[idx_high_deaths]

In [ ]:
## plotting example force plot for most inf.deaths (from linreg)
shap.force_plot(explainer_linreg.expected_value, 
                shap_values=shap_values_linreg[idx_high_deaths].values,
               features=X_shap_local.iloc[idx_high_deaths])

In [ ]:
## plotting example force plot for most inf.deaths (from rf)
shap.force_plot(explainer.expected_value, 
                shap_values=shap_values[idx_high_deaths].values,
               features=X_shap_local.iloc[idx_high_deaths])

> What do you notice when comparing the lin reg and rf reg force plots?

In [ ]:
shap.force_plot(explainer_linreg.expected_value,shap_values.values,X_shap_local,)

### Global Force Plots

In [ ]:
shap.force_plot(explainer.expected_value,shap_values.values,X_shap_local)

# LIME

In [ ]:
from lime.lime_tabular import LimeTabularExplainer
lime_explainer =LimeTabularExplainer(
    training_data=X_shap_local.values,  
    feature_names=X_shap_local.columns,
   mode='regression'
)
lime_explainer

In [ ]:
exp = lime_explainer.explain_instance(X_shap_local.loc[idx_high_deaths],
                                      rf_reg.predict)
exp.show_in_notebook()